In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show selenium

Name: seleniumNote: you may need to restart the kernel to use updated packages.

Version: 4.28.1
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: c:\practice\.venv\Lib\site-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 


In [3]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get("http://www.google.com")
print(driver.title)
driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x00007FF7ED3E02F5+28725]
	(No symbol) [0x00007FF7ED342AE0]
	(No symbol) [0x00007FF7ED1D510A]
	(No symbol) [0x00007FF7ED1AEEA5]
	(No symbol) [0x00007FF7ED256F87]
	(No symbol) [0x00007FF7ED26FA52]
	(No symbol) [0x00007FF7ED24FD53]
	(No symbol) [0x00007FF7ED21A0E3]
	(No symbol) [0x00007FF7ED21B471]
	GetHandleVerifier [0x00007FF7ED70F30D+3366989]
	GetHandleVerifier [0x00007FF7ED7212F0+3440688]
	GetHandleVerifier [0x00007FF7ED7178FD+3401277]
	GetHandleVerifier [0x00007FF7ED4AAAAB+858091]
	(No symbol) [0x00007FF7ED34E74F]
	(No symbol) [0x00007FF7ED34A304]
	(No symbol) [0x00007FF7ED34A49D]
	(No symbol) [0x00007FF7ED338B69]
	BaseThreadInitThunk [0x00007FFA59F57374+20]
	RtlUserThreadStart [0x00007FFA5B53CC91+33]


In [ ]:
pip install webdriver-manager

In [60]:
pip install --upgrade webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 통합_ver3

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv

def get_product_list_with_scroll(url):
    """스크롤을 내려가며 제품 목록을 가져오는 함수"""
    # Chrome 설정 (Headless 모드로 설정하여 브라우저를 화면에 띄우지 않음)
    options = Options()
    options.headless = False  # 화면을 띄워서 테스트 (필요에 따라 True로 설정)
    
    # ChromeDriver 설정
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # 페이지 로딩 대기
    time.sleep(3)

    # 제품 목록이 들어있는 div 요소 로딩 대기
    product_list_div = driver.find_element(By.CSS_SELECTOR, '.productCardList.module__grid')

    # 스크롤을 내리며 더 많은 제품을 로드
    last_height = driver.execute_script("return document.body.scrollHeight")
    products = []
    seen_urls = set()  # 중복을 체크하기 위한 세트

    while True:
        # 현재 페이지 소스
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 제품 카드 가져오기
        items = soup.select('.productCard.shop-productCard.typeSmall')
        for item in items:
            product_info = extract_product_info(item)  # 제품 정보 추출
            if product_info:
                detail_url = product_info["상세 URL"]
                if detail_url not in seen_urls:  # 중복 체크
                    products.append(product_info)
                    seen_urls.add(detail_url)  # 세트에 URL 추가

        # 페이지 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 로딩 대기

        # 새로운 높이를 계산하여 이전 높이와 비교
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 더 이상 스크롤할 곳이 없으면 종료

        last_height = new_height

    driver.quit()
    return products

def extract_product_info(item):
    """제품 정보를 추출하는 함수 (상세정보 포함)"""
    try:
        product_name = item.find('a', class_='prodCardBody')['ap-prd-name']
        price = item.find('a', class_='prodCardBody')['ap-prd-price']
        detail_url = "https://www.amoremall.com" + item.find('a', class_='prodCardBody')['href']

        image_tag = item.find('img')
        if image_tag:
            image_url = image_tag['src'].split("?")[0]  # 이미지 URL 정제
        else:
            image_url = ""  # 이미지가 없는 경우 빈 문자열 처리

        # 상세 URL을 사용하여 설명과 성분을 추출
        description, ingredients = extract_detail_info_with_selenium(detail_url)

        return {
            "제품명": product_name,
            "설명": description,
            "가격": price,
            "이미지 URL": image_url,
            "상세 URL": detail_url,
            "성분": ingredients
        }
    except AttributeError as e:
        print(f"Error extracting product info: {e}")
        return None

def extract_detail_info_with_selenium(detail_url):
    """Selenium을 이용해 제품 상세 페이지에서 설명과 성분을 추출하는 함수"""
    # Chrome 설정 (Headless 모드로 설정하여 브라우저를 화면에 띄우지 않음)
    options = Options()
    options.headless = True

    # ChromeDriver 설정
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(detail_url)

    # 페이지 로딩 대기 (필요한 시간만큼 대기)
    time.sleep(3)  # 페이지가 로드될 때까지 기다림 (필요에 따라 조정)

    # HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()  # 드라이버 종료

    description = ""
    ingredients = ""

    # <div class="infoDetailWrap"> 안에서 dSection을 찾아 원하는 정보 추출
    info_detail_wrap = soup.select_one('.infoDetailWrap')
    if not info_detail_wrap:
        print("❌ [오류] infoDetailWrap을 찾을 수 없음.")
        return "", ""

    for d_section in info_detail_wrap.find_all('div', class_='dSection'):
        head = d_section.find('h4', class_='head small')
        if head:
            if "제품 소개" in head.text.strip():
                description = "\n".join(p.text.strip() for p in d_section.find_all('p', class_='txtNotice dotted'))
            elif "｢화장품법｣에 따라 기재ㆍ표시하여야 하는 모든 성분" in head.text.strip():
                ingredients = "\n".join(p.text.strip() for p in d_section.find_all('p', class_='txtNotice dotted'))

    if not description:
        print("⚠️ [경고] 제품 소개 정보를 찾을 수 없음.")
    if not ingredients:
        print("⚠️ [경고] 성분 정보를 찾을 수 없음.")

    return description, ingredients

def save_to_csv(product_list, filename="sulwhasoo_all.csv"):
    """제품 정보를 CSV 파일에 저장하는 함수"""
    if not product_list:
        return

    with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = product_list[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(product_list)

if __name__ == "__main__":
    base_url = "https://www.amoremall.com/kr/ko/display/brand/detail/all?brandSn=18"
    products = get_product_list_with_scroll(base_url)

    if products:
        print(f"Found {len(products)} sulwhasoo_all.")
        save_to_csv(products)
        print(f"Product information saved to sulwhasoo_all.csv")
    else:
        print("Failed to retrieve product information.")
